START --> LLM_QA [User asks questions, LLM generates Answer, State gets updated with the answer] --> END

In [28]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [29]:
load_dotenv()

True

In [30]:
model = ChatOpenAI(model="gpt-4o-mini")

In [31]:
#create a state
class LLMState(TypedDict):

    question: str
    answer: str

In [32]:
def llm_qa(state: LLMState) -> LLMState:

    #extract the question from state
    question = state['question']

    #form a prompt
    prompt = f'Answer the following question {question}'

    # ask that question to the LLM
    answer = model.invoke(prompt).content

    # update the answer in the state
    state['answer'] = answer

    return state

In [33]:
# create our graph

graph = StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa', llm_qa)

#add edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

#compile
workflow = graph.compile()


In [34]:
#execute

initial_state = {'question':'How far is moon from earth'}

final_state = workflow.invoke(initial_state)

print(final_state)

{'question': 'How far is moon from earth', 'answer': "The average distance from the Earth to the Moon is about 238,855 miles (384,400 kilometers). However, this distance can vary slightly due to the elliptical shape of the Moon's orbit, ranging from approximately 225,623 miles (363,104 kilometers) at its closest (perigee) to about 252,088 miles (405,696 kilometers) at its farthest (apogee)."}


In [35]:
model.invoke('How far is moon from earth').content

"The average distance from the Earth to the Moon is about 238,855 miles (384,400 kilometers). However, this distance can vary slightly due to the Moon's elliptical orbit, ranging from approximately 225,623 miles (363,104 kilometers) at its closest (perigee) to about 252,088 miles (405,696 kilometers) at its farthest (apogee)."